In [19]:
import numpy as np 
import pandas as pd
import string 
import nltk
import re
from bs4 import BeautifulSoup
import requests
from string import punctuation
stopword = nltk.corpus.stopwords.words('english')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

wn = nltk.WordNetLemmatizer()
le = preprocessing.LabelEncoder()

In [ ]:
url = data.iloc[:,2]
lyric_list = []
for URL in url:
    page = requests.get(URL)
    html = BeautifulSoup(page.text, "html.parser") # Extract the page's HTML as a string

    # Scrape the song lyrics from the HTML
    lyrics = html.find("div", class_="lyrics").get_text()
    lyric_list.append(lyrics)
%store lyric_list

In [3]:
%store -r lyric_list
print(len(lyric_list))

2022


# BASELINE APPROACH

# PROJECT CODE

In [5]:
data = pd.read_csv('final_song_data.csv',delimiter=',',header =None)
artist_name = data.iloc[:,0]
song_name = data.iloc[:,1]
data_set = [list(data_row) for data_row in zip(artist_name,song_name,lyric_list)]
labels = le.fit_transform(data.iloc[:,3])

In [6]:
def process_features(data_set):
    
    lyric_list_new = []
    artist_name_new = []
    song_name_new = []
    chorus_list = []
    verse_list = []
    bridge_list = []
    hook_list = []
    intro_list = []
    outro_list = []
    song_length = []
    for dat in data_set:
        s = dat[2]
        chorus_list.append(len(re.findall('\[Chorus', s)))
        verse_list.append(len(re.findall('\[Verse', s)))
        hook_list.append(len(re.findall('\[Hook', s)))
        bridge_list.append(len(re.findall('\[Bridge', s)))
        intro_list.append(len(re.findall('\[Intro', s)))
        outro_list.append(len(re.findall('\[Outro', s)))

        v = s.split("\n")
        song_length.append(len(v))
        for i in v:
            d = re.sub("^\[(.*)]","",i)
            v[v.index(i)] = d
        lyric_list_new.append("\n".join(v))       
        
        artist_name_new.append(dat[0])
        song_name_new.append(dat[1])

   
    for element in range(1,len(lyric_list_new)): 
        
        #LYRICS
        lyric_list_new[element] =lyric_list_new[element].replace("'"," ")
        lyric_list_new[element] = re.sub(r'[^\w\s]','',lyric_list_new[element])
        lyric_list_new[element] = nltk.word_tokenize(lyric_list_new[element])
        not_stopword_list = [word.lower() for word in lyric_list_new[element] if word.lower() not in stopword]
        t = " ".join([wn.lemmatize(word) for word in not_stopword_list])
        lyric_list_new[element]=t
        
        #ARTIST
        artist_name_new[element] =artist_name_new[element].replace(",","")
        artist_name_new[element] =artist_name_new[element].replace("&","")

        
        #SONG TITLE
        song_name_new[element] =song_name_new[element].replace("'"," ")
        song_name_new[element] = re.sub(r'[^\w\s]','',song_name_new[element])
        song_name_new[element] = nltk.word_tokenize(song_name_new[element])
        not_stopword_list = [word.lower() for word in song_name_new[element] if word.lower() not in stopword]
        t = " ".join([wn.lemmatize(word) for word in not_stopword_list])
        song_name_new[element]=t
        
    data_set_new = [artist_name_new,song_name_new,lyric_list_new]
    lyric_all = np.vstack((np.array(chorus_list),np.array(verse_list),np.array(bridge_list),np.array(hook_list),np.array(intro_list),np.array(outro_list),np.array(song_length)))
    lyric_all = np.transpose(lyric_all)
    return data_set_new,lyric_all

In [15]:
x_train, x_test, y_train, y_test = train_test_split(data_set,labels, train_size=0.8,test_size = 0.2)

#Training Set 
dataset_train_sub,feat_add_train = process_features(x_train)
artist_train_sub = dataset_train_sub[0]
song_title_train_sub = dataset_train_sub[1]
lyric_train_sub = dataset_train_sub[2]
lyric_train_label = y_train


#Test set 
dataset_test_sub,feat_add_test = process_features(x_test)
artist_test_sub = dataset_test_sub[0]
song_title_test_sub = dataset_test_sub[1]
lyric_test_sub = dataset_test_sub[2]
lyric_test_label = y_test




['182', '1975', '21', '2pac', '40', '50', '6ix9ine', '6lack', 'ab', 'abstract', 'accept', 'ace', 'action', 'adele', 'after', 'again', 'against', 'aiko', 'alan', 'alessia', 'aloe', 'alt', 'alter', 'am', 'amarante', 'ambassadors', 'aminé', 'amy', 'and', 'andrew', 'animals', 'anne', 'aoki', 'ap', 'appeal', 'archies', 'arctic', 'ariana', 'armstrong', 'arthur', 'astaire', 'at', 'atari', 'atb', 'attractions', 'avicii', 'axwell', 'ayo', 'azalea', 'azealia', 'baby', 'backstreet', 'bacon', 'bad', 'baddiel', 'balvin', 'bandit', 'bangles', 'bangz', 'banks', 'barry', 'bastard', 'battalion', 'battles', 'beal', 'beard', 'beastie', 'beatles', 'bebe', 'belle', 'bellion', 'ben', 'benny', 'bethel', 'beyonce', 'beyoncé', 'bhabie', 'bhad', 'bieber', 'big', 'billie', 'billy', 'bishop', 'bizarre', 'bizkit', 'blacc', 'black', 'blackbear', 'blake', 'blanco', 'bleu', 'blink', 'blocboy', 'bloodpop', 'blue', 'bob', 'bobby', 'bon', 'bone', 'boogie', 'boomin', 'boomtown', 'boy', 'boys', 'boyz', 'brandon', 'brian',

In [36]:
#FEATURE EXTRACTION
C_vec = CountVectorizer(max_df = 0.5,min_df = 2,ngram_range = (1,2))
trainX1 = C_vec.fit_transform(lyric_train_sub).toarray()
testX1 = C_vec.transform(lyric_test_sub).toarray()
# print(C_vec.get_feature_names())

C_vec = CountVectorizer(ngram_range = (1,3))
trainX2 = C_vec.fit_transform(artist_train_sub).toarray()
testX2 = C_vec.transform(artist_test_sub).toarray()
print(C_vec.get_feature_names())

C_vec = CountVectorizer(max_df = 0.5,min_df = 2,ngram_range = (1,2))
trainX3 = C_vec.fit_transform(song_title_train_sub).toarray()
testX3 = C_vec.transform(song_title_test_sub).toarray()
print(C_vec.get_feature_names())

lyric_train = np.hstack((trainX1,trainX2,trainX3,feat_add_train))
lyric_test = np.hstack((testX1,testX2,testX3,feat_add_test))


#Final feature : lyric_train
print("Train Data Shape: ",np.shape(lyric_train))
print("Test Data Shape: ",np.shape(lyric_test))

['182', '1975', '21', '21 savage', '21 savage metro', '21 savage offset', '2pac', '40', '50', '50 cent', '6ix9ine', '6lack', 'ab', 'ab soul', 'abstract', 'accept', 'ace', 'ace hood', 'action', 'action bronson', 'adele', 'after', 'after sex', 'again', 'against', 'against the', 'against the machine', 'aiko', 'alan', 'alan walker', 'alessia', 'alessia cara', 'aloe', 'aloe blacc', 'alt', 'alter', 'alter bridge', 'am', 'amarante', 'ambassadors', 'ambassadors bebe', 'ambassadors bebe rexha', 'aminé', 'amy', 'amy winehouse', 'and', 'and guy', 'and guy sebastian', 'and kolacny', 'and kolacny brothers', 'and mary', 'and mary chain', 'andrew', 'andrew belle', 'andrew belle featuring', 'animals', 'anne', 'anne marie', 'aoki', 'aoki louis', 'aoki louis tomlinson', 'ap', 'ap ferg', 'ap mob', 'ap rocky', 'appeal', 'appeal records', 'archies', 'arctic', 'arctic monkeys', 'ariana', 'ariana grande', 'armstrong', 'arthur', 'astaire', 'at', 'at the', 'at the disco', 'atari', 'atari teenage', 'atari teena

Train Data Shape:  (1617, 8338)
Test Data Shape:  (405, 8338)


In [48]:
#FEATURE EXTRACTION
T_vec = TfidfVectorizer(max_df = 0.5,min_df = 1,ngram_range = (1,2))
trainX1_tf = T_vec.fit_transform(lyric_train_sub).toarray()
testX1_tf = T_vec.transform(lyric_test_sub).toarray()

T_vec = TfidfVectorizer(ngram_range = (1,2))
trainX2_tf = T_vec.fit_transform(artist_train_sub).toarray()
testX2_tf = T_vec.transform(artist_test_sub).toarray()


T_vec = TfidfVectorizer(max_df = 0.5,min_df = 1,ngram_range = (1,2))
trainX3_tf = T_vec.fit_transform(song_title_train_sub).toarray()
testX3_tf = T_vec.transform(song_title_test_sub).toarray()

lyric_train = np.hstack((trainX1_tf,trainX2_tf,trainX3_tf,feat_add_train))
lyric_test = np.hstack((testX1_tf,testX2_tf,testX3_tf,feat_add_test))


#Final feature : lyric_train
print("Train Data Shape: ",np.shape(lyric_train))
print("Test Data Shape: ",np.shape(lyric_test))

Train Data Shape:  (1617, 240894)
Test Data Shape:  (405, 240894)


# CLASSIFIERS

In [49]:
## MULTINOMIAL NAIVE BAYES

nb_g = GaussianNB()
model =nb_g.fit(lyric_train,lyric_train_label)
lyric_train_label_predict = model.predict(lyric_train)
accuracy_train = accuracy_score(lyric_train_label,lyric_train_label_predict)
lyric_test_label_predict = model.predict(lyric_test)
accuracy_test = accuracy_score(lyric_test_label,lyric_test_label_predict)
# print(lyric_test_label,lyric_test_label_predict)
print('GaussianNB_Accuracy_Train = ',accuracy_train)
print('GaussianNB_Accuracy_Test = ',accuracy_test)
print('F1_Score = ',f1_score(lyric_test_label,lyric_test_label_predict,average = 'macro') )

GaussianNB_Accuracy_Train =  1.0
GaussianNB_Accuracy_Test =  0.5012345679012346
F1_Score =  0.49518481499025163


In [42]:
## SUPPORT VECTOR MACHINES

clf = SVC()
model = clf.fit(lyric_train,lyric_train_label)
lyric_train_label_predict = model.predict(lyric_train)
accuracy_train = accuracy_score(lyric_train_label,lyric_train_label_predict)
lyric_test_label_predict = model.predict(lyric_test)
accuracy_test = accuracy_score(lyric_test_label,lyric_test_label_predict)
# print(lyric_test_label,lyric_test_label_predict)
print('SVM_Accuracy_Train = ',accuracy_train)
print('SVM_Accuracy_Test = ',accuracy_test)

SVM_Accuracy_Train =  0.6431663574520717
SVM_Accuracy_Test =  0.5160493827160494


In [47]:
## LOGISTICAL REGRESSION

#Cross Validation for LogisticRegression

feat_train,x_validation,feat_label,label_validation = train_test_split(lyric_train,lyric_train_label,test_size = 0.25,shuffle = True)
max_acc = 0
for c in np.linspace(0.001,1000,10):
    for p in ['l1','l2']:

        Lregression = LogisticRegression(multi_class = 'ovr',C=c,penalty = p)
        OneVRest = OneVsRestClassifier(Lregression, n_jobs=-1)
        model = OneVRest.fit(feat_train,feat_label)
        label_validation_predict = model.predict(x_validation)
        accuracy = accuracy_score(label_validation,label_validation_predict)
        print('C := ',c, '  Penalty:= ',p,'  Accuracy= ',accuracy)
        if(accuracy>=max_acc):
            max_acc = accuracy
            optimal_C = c
            optimal_P = p

Lregression = LogisticRegression(multi_class = 'ovr',C = optimal_C,penalty = optimal_P)
OneVRest = OneVsRestClassifier(Lregression, n_jobs=-1)
model = OneVRest.fit(feat_train,feat_label)
label_validation_predict = model.predict(x_validation)
accuracy = accuracy_score(label_validation,label_validation_predict)
print('Accuracy= ',accuracy) 

C :=  0.001   Penalty:=  l1   Accuracy=  0.4222222222222222
C :=  0.001   Penalty:=  l2   Accuracy=  0.4222222222222222
C :=  111.11200000000001   Penalty:=  l1   Accuracy=  0.4740740740740741
C :=  111.11200000000001   Penalty:=  l2   Accuracy=  0.5160493827160494
C :=  222.223   Penalty:=  l1   Accuracy=  0.4740740740740741
C :=  222.223   Penalty:=  l2   Accuracy=  0.5185185185185185
C :=  333.334   Penalty:=  l1   Accuracy=  0.47901234567901235
C :=  333.334   Penalty:=  l2   Accuracy=  0.5185185185185185
C :=  444.445   Penalty:=  l1   Accuracy=  0.49135802469135803
C :=  444.445   Penalty:=  l2   Accuracy=  0.5160493827160494
C :=  555.556   Penalty:=  l1   Accuracy=  0.48641975308641977
C :=  555.556   Penalty:=  l2   Accuracy=  0.5135802469135803
C :=  666.667   Penalty:=  l1   Accuracy=  0.5012345679012346
C :=  666.667   Penalty:=  l2   Accuracy=  0.5135802469135803
C :=  777.778   Penalty:=  l1   Accuracy=  0.49382716049382713
C :=  777.778   Penalty:=  l2   Accuracy=  0.513

In [50]:
## RANDOM FOREST

feat_train,x_validation,feat_label,label_validation = train_test_split(lyric_train,lyric_train_label,test_size = 0.25,shuffle = True)
max_acc = 0
for n in range(1,31):
        rfc = RandomForestClassifier(n_estimators = n)
        model = rfc.fit(feat_train, feat_label)
        label_validation_predict = model.predict(x_validation)
        accuracy = accuracy_score(label_validation,label_validation_predict)
        if(accuracy>=max_acc):
            max_acc = accuracy
            optimal_n = n
rfc = RandomForestClassifier(n_estimators = optimal_n)
model = rfc.fit(lyric_train, lyric_train_label)
lyric_train_label_predict = model.predict(lyric_train)
accuracy_train = accuracy_score(lyric_train_label,lyric_train_label_predict)
lyric_test_label_predict = model.predict(lyric_test)
accuracy_test = accuracy_score(lyric_test_label,lyric_test_label_predict)
print('RF_Accuracy_Train = ',accuracy_train)
print('RF_Accuracy_Test = ',accuracy_test)

RF_Accuracy_Train =  0.9962894248608535
RF_Accuracy_Test =  0.4839506172839506


In [ ]:
## CONVOLUTIONAL NEURAL NETWORKS
import tensorflow
import keras
import numpy as np
from matplotlib import pyplot as plt 
import keras
import sys
import math

from keras.utils import to_categorical
np.random.seed(123)
import tensorflow as tf
tf.set_random_seed(123)
from keras.models import Sequential
from keras.layers import Dense,Dropout, Activation,Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras import optimizers
from keras.callbacks import LearningRateScheduler


##(feat_train,l_train),(feat_test,l_test) = data
##feat_train = feat_train.reshape(feat_train.shape[0],28,28,1)
##feat_test = feat_test.reshape(feat_test.shape[0],28,28,1)
feat_train = lyric_train.astype('float32')
feat_test = lyric_test.astype('float32')
##feat_train/=255
##feat_test/=255
label_train = to_categorical(lyric_train_label,3)
label_test = to_categorical(lyric_test_label,3)
feat_train_shape = feat_train.shape
feat_test_shape = feat_test.shape
print(feat_train_shape,len(feat_train))
print(feat_test_shape)
print(label_train.shape)
print(label_test.shape)

nb_features = int(feat_train_shape[1]/3)
# number of features per features type (shape, texture, margin)   
classes = [0,1,2]
nb_class = 3

# reshape train data
# feat_train_r = np.zeros((len(feat_train), nb_features, 3))
# feat_train_r[:, :, 0] = feat_train[:, :nb_features]
# feat_train_r[:, :, 1] = feat_train[:, nb_features:(2*nb_features)+1]
# feat_train_r[:, :, 2] = feat_train[:, (2*nb_features)+1:]

# # reshape test data
# feat_test_r = np.zeros((len(feat_test), nb_features, 3))
# feat_test_r[:, :, 0] = feat_test[:, :nb_features]
# feat_test_r[:, :, 1] = feat_test[:, nb_features:2830]
# feat_test_r[:, :, 2] = feat_test[:, 2830:]

feat_train = feat_train.reshape(feat_train.shape[0],142,149).astype('float32')
feat_test = feat_test.reshape(feat_test.shape[0],142,149).astype('float32')

# cnn_model = Sequential()

# cnn_model.add(Conv1D(6, 5,padding='same', activation='relu', use_bias=True,input_shape = (feat_train_shape[0]
#                                                                                           ,feat_train_shape[1]), kernel_initializer = 'RandomUniform'))
# # print(cnn_model.output_shape)

# # cnn_model.add(MaxPooling1D(pool_size=2))
# # print(cnn_model.output_shape)

# cnn_model.add(Conv1D(16, 5,padding='valid', activation='relu', use_bias=True))
# # print(cnn_model.output_shape)

# # cnn_model.add(MaxPooling1D(pool_size=2))
# # print(cnn_model.output_shape)

# cnn_model.add(Flatten())
# cnn_model.add(Dense(120, activation='relu', use_bias=True))
# cnn_model.add(Dropout(0.2))
# # print(cnn_model.output_shape)
# cnn_model.add(Dense(80, activation='relu', use_bias=True))
# # print(cnn_model.output_shape)
# cnn_model.add(Dense(10, activation='softmax'))
# # print(cnn_model.output_shape)

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=10, input_shape=(142,149),activation='relu'))
model.add(MaxPooling1D(pool_size=5 ))
model.add(Conv1D(filters =64, kernel_size = 8,padding='valid', activation='relu', use_bias=True))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1000, activation='relu',use_bias=True))
model.add(Dropout(0.5))
model.add(Dense(80, activation='relu', use_bias=True))
model.add(Dense(3, activation='softmax'))


# #lr *= (1. / (1. + self.decay * self.iterations))
learning_rate = 0.05
decay_rate = learning_rate / 100
momentum = 0.8
# sgd = SGD(lr=0.01, nesterov=True, decay=1e-6, momentum=0.9)
# model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

sgd = optimizers.SGD(lr = learning_rate,momentum = momentum,decay=decay_rate)
model.compile(loss='categorical_crossentropy',optimizer= sgd,metrics=['accuracy'])

history = model.fit(feat_train,label_train,epochs = 100, verbose = 2,validation_data = (feat_test,label_test))
performance = model.evaluate(feat_test,label_test,verbose = 0)
print('Test Loss:', performance[0])
print('Test Accuracy: ', performance[1])
performance = model.evaluate(feat_train,label_train,verbose = 0)
print('Train Loss:', performance[0])
print('Train Accuracy: ', performance[1])

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()